# Tuning the value of K using polarizablity.


#### path = "............" refers to the path of the dump file from which data is to be extracted

#### frames = ------ number of frames during openmd execution

#### atomNumber = ----------- total number of atomic sites during program execution

#### atomPlate= ................ number of atoms in capacitave plates

In [1]:
# extract information from dump file.
import numpy as num
import matplotlib.pyplot as plt
import pylab as lab
from scipy import constants
import scipy as sci
import pandas as pan
import os
from collections import OrderedDict
from scipy.optimize import curve_fit



%matplotlib inline

In [2]:

    
"""
infoDict=DumpExtractor(filename,frames,atomNumber,atomPlate)


Function that extracts the information from the .dump file created by openmd
    
    
    Inputs:
  ===========
   
   
   filename:
   
               Path of the dump file from which the information is to be extracted
               
    frame:
    
                Total number of frames in the dump file
                
    atomNumber:
        
                Totla number of atoms in the slab or crystal
                
    atomPlate:
    
                Total number of atoms in the capacitor plates



    Outputs:
 =============
 
 infoDict:
 
         Dictonary containing position, velocity, chargeQV, electricField, plateEQV.
         Postion is a list of [x,y,z] and each x,y,z are array of x[frames][sites]
         velocity is a list of [vx,vy,vz] and each vx,vy,vz are array of vx[frames][sites]
         chargeQV is a lisf of [c,cv] and each c and cv are array of c[frame][sites]
         electric field is list of [ex,ey,ez] and each are array of ex[frame][sites]
         plateEQV is the list of [pex,pey,pez,pc,pcv] and each are array of pex[frames][sites]
"""
   
def DumpExtractor(filename,frames,atomNumber,atomPlate):
    
    fileDump=open(filename)  #dump file for info extraction
    linesDump=fileDump.readlines()

    processP="Wait"
    processC="Wait"


    #information storage matrix 
    #posiiton and velocity storage
    x=num.zeros((frames,atomNumber+1))
    y=num.zeros((frames,atomNumber+1))
    z=num.zeros((frames,atomNumber+1))
    vx=num.zeros((frames,atomNumber+1))
    vy=num.zeros((frames,atomNumber+1))
    vz=num.zeros((frames,atomNumber+1))


    #charge and velocity storage matrix
    c=num.zeros((frames,atomNumber+1))
    cv=num.zeros((frames,atomNumber+1))
    ex=num.zeros((frames,atomNumber+1))
    ey=num.zeros((frames,atomNumber+1))
    ez=num.zeros((frames,atomNumber+1))
    pc=num.zeros((frames,atomPlate))
    pcv=num.zeros((frames,atomPlate))
    pex=num.zeros((frames,atomPlate))
    pey=num.zeros((frames,atomPlate))
    pez=num.zeros((frames,atomPlate))

    #frame count initilization
    fCount=0
    index=0  #index for the atoms
    for line in linesDump:
        linesSplit=str.split(line)
        length=len(linesSplit)
    
        if(length!=0 and linesSplit[0]=="<StuntDoubles>" and processP=="Wait"):
            processP="Start"
            continue;
        
        elif(length!=0 and linesSplit[0]=="</StuntDoubles>" and processP=="Start"):
            processP="Wait"
            index=0
            continue;
        
        elif(length!=0 and linesSplit[0]=="<SiteData>" and processC=="Wait"):
            processC="Start"
            continue;
        
        elif(length!=0 and linesSplit[0]=="</SiteData>" and processC=="Start"):
            fCount=fCount+1
            index=0;
            processC="Wait"
            continue;
   
        elif(fCount>=frames):
            break;
        
        else:
            processP=processP;
            processC=processC;
        
        
        if (processP=="Start"):
            x[fCount][int(linesSplit[0])]=float(linesSplit[2])
            y[fCount][int(linesSplit[0])]=float(linesSplit[3])
            z[fCount][int(linesSplit[0])]=float(linesSplit[4])
            vx[fCount][int(linesSplit[0])]=float(linesSplit[5])
            vy[fCount][int(linesSplit[0])]=float(linesSplit[6])
            vz[fCount][int(linesSplit[0])]=float(linesSplit[7])
        
        if(processC=="Start"):
            if int(linesSplit[0])<atomNumber:
                c[fCount][int(linesSplit[0])]=float(linesSplit[3])
                cv[fCount][int(linesSplit[0])]=float(linesSplit[4])
                ex[fCount][int(linesSplit[0])]=float(linesSplit[5])
                ey[fCount][int(linesSplit[0])]=float(linesSplit[6])
                ez[fCount][int(linesSplit[0])]=float(linesSplit[7])
            elif (int(linesSplit[0])==atomNumber and linesSplit[1]=="cwe"):
                continue
                c[fCount][int(linesSplit[0])]=float(linesSplit[2])
                cv[fCount][int(linesSplit[0])]=float(linesSplit[3])
                ex[fCount][int(linesSplit[0])]=float(linesSplit[4])
                ey[fCount][int(linesSplit[0])]=float(linesSplit[5])
                ez[fCount][int(linesSplit[0])]=float(linesSplit[6])
            else:
                pc[fCount][int(linesSplit[1])]=float(linesSplit[3])
                pcv[fCount][int(linesSplit[1])]=float(linesSplit[4])
                pex[fCount][int(linesSplit[1])]=float(linesSplit[5])
                pey[fCount][int(linesSplit[1])]=float(linesSplit[6])
                pez[fCount][int(linesSplit[1])]=float(linesSplit[7])
        
    position=[x,y,z]
    velocity=[vx,vy,vz]
    chargeQV=[c,cv]
    electricField=[ex,ey,ez]
    platesEQV=[pex,pey,pez,pc,pcv]
    
    infoDict={"position":position,"velocity":velocity,"chargeQV":chargeQV,"electricField":electricField,"platesEQV":platesEQV}
    return infoDict


In [3]:
"""Function that determines different layers in a crystal

[layer,a]= Layers(ZPosition,atomNumber)
 
  Input:
 ========
 
         ZPosition: Z Coordinates of lattice for layer determination
         
         atomNumber: total Number of atoms in crystal
         
         
  Output:
 =========
         list [layer,a]; layer has index for atoms in each layers and "a" has the z-coordinates for each layers
         
         
"""
def Layers(ZPosition,atomNumber):
    a=num.sort(list(set(ZPosition[0,0:atomNumber-1])))
    layer=[]
    for var in a:
        layer.append(num.where(ZPosition[0]==var))
    
    return [layer,a]

In [4]:
def LayerDipole(dumpFile,frames,atomNumber,atomPlate,UsedFrame,K,E,plotBool):
    try:
        dump=DumpExtractor(dumpFile,frames,atomNumber,atomPlate)
        print("dump")
        pos=dump["position"]
        charge=dump["chargeQV"]
        layersInfo=Layers(pos[2],atomNumber)
        layer=layersInfo[0]
        a=layersInfo[1]
        averageChargeLayers=[]
        aveZpos=[]
    
    
        for counter in range(len(a)):
            totalData=float(len(layer[counter][0]))*(frames-UsedFrame)
            averageChargeLayers.append(num.sum(charge[0][UsedFrame:,layer[counter][0]])/totalData)
            aveZpos.append(num.sum(pos[2][UsedFrame:,layer[counter][0]])/totalData)
        
        
        
        diff=[]
        precharge=0
        prez=0
        for counter in range(len(a)):
            diff.append((averageChargeLayers[counter]-precharge)*(aveZpos[counter]-prez))
            precharge=averageChargeLayers[counter]
            prez=aveZpos[counter]
    
         
        if plotBool==True:
            lab.plot(range(1,len(a)),diff[1:],'o-')
            lab.xlabel("Layers")
            lab.ylabel("LayerDipole")
            lab.title("E = "+str(E)+" || K = "+str(K))
            lab.grid()
            lab.show()
        dipole=num.sum(diff[4:-4])/float(len(diff)-9)
        #dipole=num.sum(diff[4:-4])/float(len(diff)-9)
        
        return dipole
    
        
    except:
        print("Corrupt File")
        return 999999
    
    
    

In [5]:
def LayerFitDipole(dumpFile,frames,atomNumber,atomPlate,UsedFrame,K,E,plotBool):
    try:
        dump=DumpExtractor(dumpFile,frames,atomNumber,atomPlate)
        print("dump")
        pos=dump["position"]
        charge=dump["chargeQV"]
        layersInfo=Layers(pos[2],atomNumber)
        layer=layersInfo[0]
        a=layersInfo[1]
        averageChargeLayers=[]
        aveZpos=[]
    
    
        for counter in range(len(a)):
            totalData=float(len(layer[counter][0]))*(frames-UsedFrame)
            averageChargeLayers.append(num.sum(charge[0][UsedFrame:,layer[counter][0]])/totalData)
            aveZpos.append(num.sum(pos[2][UsedFrame:,layer[counter][0]])/totalData)
        
        n=num.arange(1,len(a)+1-8)
        if len(a)%2==0:
            n=n-num.ceil((len(a)+1)/2)  # shfting 1 2 3 4 5 6 to -3 -2 -1 1 2 3
            n[n>=0]=n[n>=0]+1
        else:
            n=n-num.ceil(len(a)/2)
            
        
            
        fitFx=lambda x,a:a*x
        paramCharge=curve_fit(fitFx,n,averageChargeLayers[4:-4])
        paramZPos=curve_fit(fitFx,n,aveZpos[4:-4])
        
        averageChargeFitted=fitFx(n,paramCharge[0])
        averageZPosFitted=fitFx(n,paramZPos[0])
        
        lab.plot(averageChargeLayers[4:-4])
        lab.plot(averageChargeFitted)
        
        diff=[]
        precharge=0
        prez=0
        for counter in range(len(a)):
            diff.append((averageChargeFitted[counter]-precharge)*(averageZPosFitted[counter]-prez))
            precharge=averageChargeFitted[counter]
            prez=averageZPosFitted[counter]
    
         
        if plotBool==True:
            lab.plot(range(1,len(a)),diff[1:],'o-')
            lab.xlabel("Layers")
            lab.ylabel("LayerDipole")
            lab.title("E = "+str(E)+" || K = "+str(K))
            lab.grid()
            lab.show()
        dipole=num.sum(diff[4:-4])/float(len(diff)-9)
        print(dipole)
        #dipole=num.sum(diff[4:-4])/float(len(diff)-9)
        
        return dipole
    
        
    except:
        print("Corrupt File")
        

In [6]:
def SlabDipole(dumpFile,frames,atomNumber,atomPlate,UsedFrame):
    try:
        dump=DumpExtractor(dumpFile,frames,atomNumber,atomPlate)
        pos=dump["position"]
        charge=dump["chargeQV"]
        z=pos[2][UsedFrame,:]
        c=charge[0][UsedFrame,:]
        dipole=num.sum(num.multiply(z,c))
        return dipole
    
    except:
        print("Corrupt File")
        return 999999
    

In [7]:
def ChargeVZPos(dumpFile,frames,UsedFrame,atomNumber,atomPlate,K,E,plotBool):
    try:
        dump=DumpExtractor(dumpFile,frames,atomNumber,atomPlate)
        pos=dump["position"]
        charge=dump["chargeQV"]
        layersInfo=Layers(pos[2],atomNumber)
        layer=layersInfo[0]
        a=layersInfo[1]
        averageChargeLayers=[]
        averagePos=[]
        
        
        if plotBool==True:
            colors=['b','b--', 'g--','g','r--','r','c--','c','m--','m','y--','y'\
                , 'k--','k','b-.','g-.','r-.','c-.','m-.','y-.','k-.']
            fig = plt.figure(1)
            ax = fig.add_subplot(111)
            for counter in range(len(a)):

                totalData=float(len(layer[counter][0]))*(frames-UsedFrame)
                averageChargeLayers.append(num.sum(charge[0][UsedFrame:,layer[counter][0]])/totalData)
                averagePos.append(num.sum(pos[2][:,layer[counter][0]])/totalData) 
                
            
            lab.plot(averagePos,averageChargeLayers,"o")
            #handles, labels = ax.get_legend_handles_labels()
            #lgd=lab.legend(bbox_to_anchor=(1.5,1),loc="upper right")
            lab.xlabel("Average Z Position")
            lab.ylabel("AverageCharge")
            lab.title("K = "+str(K)+"|| E="+str(E))
            lab.grid()
            lab.show()
            
    except:
        print("Corrupt file.")
    
    

In [8]:
def LayerPos(dumpFile,frames,UsedFrame,atomNumber,atomPlate,K,E,plotBool):
    try:
        dump=DumpExtractor(dumpFile,frames,atomNumber,atomPlate)
        pos=dump["position"]
        layersInfo=Layers(pos[2],atomNumber)
        layer=layersInfo[0]
        a=layersInfo[1]
        averagePos=[]
        
        
        
        if plotBool==True:
            colors=['b','b--', 'g--','g','r--','r','c--','c','m--','m','y--','y'\
                , 'k--','k','b-.','g-.','r-.','c-.','m-.','y-.','k-.']
            fig = plt.figure(1)
            ax = fig.add_subplot(111)
            
            for counter in range(len(a)):
                totalData=float(len(layer[counter][0]))*(frames-UsedFrame)
                averagePos.append(num.sum(pos[2][:,layer[counter][0]])/totalData)
            #f=lambda x:a*x+b
            #params = curve_fit(f,range(len(a)),averagePos)
            print(0)
            
            lab.plot(range(len(a)),averagePos,"o")
            #handles, labels = ax.get_legend_handles_labels()
            #lgd=lab.legend(bbox_to_anchor=(1.5,1),loc="upper right")
            lab.ylabel("Average Z Position")
            lab.xlabel("LayerNumber")
            lab.title("K = "+str(K)+"|| E="+str(E))
            lab.xlim([-1,len(a)])
            lab.show()
            #return params[0]
            
    except:
        print("Corrupt file.")
    

In [9]:
from scipy.optimize import curve_fit
def LayerCharge(dumpFile,frames,UsedFrame,atomNumber,atomPlate,K,E,plotBool):
    try:
        dump=DumpExtractor(dumpFile,frames,atomNumber,atomPlate)
        pos=dump["position"]
        charge=dump["chargeQV"]
        layersInfo=Layers(pos[2],atomNumber)
        layer=layersInfo[0]
        a=layersInfo[1]
        averageChargeLayers=[]
        
        
        
        if plotBool==True:
            colors=['b','b--', 'g--','g','r--','r','c--','c','m--','m','y--','y'\
                , 'k--','k','b-.','g-.','r-.','c-.','m-.','y-.','k-.']
            fig = plt.figure(1)
            ax = fig.add_subplot(111)
            
            for counter in range(len(a)):
                totalData=float(len(layer[counter][0]))*(frames-UsedFrame)
                averageChargeLayers.append(num.sum(charge[0][UsedFrame:,layer[counter][0]])/totalData)
            f=lambda x,a,b:a*x+b
            
            params = curve_fit(f, range(len(a)), avergaeChargeLayers)
            
            lab.plot(range(len(a)),averageChargeLayers,"o")
            lab.ylabel("Average Charge")
            lab.xlabel("LayerNumber")
            lab.title("K = "+str(K)+"|| E="+str(E))
            lab.xlim([-1,len(a)])
            lab.xticks(range(len(a)))
            lab.grid()
            lab.show()
            return params[0]
            
    except:
        print("Corrupt file.")
    
   

In [33]:
def LayerFitDipoleTest(dumpFile,frames,atomNumber,atomPlate,UsedFrame):
    try:
        dump=DumpExtractor(dumpFile,frames,atomNumber,atomPlate)
        pos=dump["position"]
        charge=dump["chargeQV"]
        layersInfo=Layers(pos[2],atomNumber)
        layer=layersInfo[0]
        a=layersInfo[1]
        averageChargeLayers=[]
        aveZpos=[]
    
    
        for counter in range(len(a)):
            totalData=float(len(layer[counter][0]))*(frames-UsedFrame)
            averageChargeLayers.append(num.sum(charge[0][UsedFrame:,layer[counter][0]])/totalData)
            aveZpos.append(num.sum(pos[2][UsedFrame:,layer[counter][0]])/totalData)
        
        n=num.arange(1,len(a)+1-8)
        if len(a)%2==0:
            n=n-num.ceil((len(a)+1-8)/2)  # shfting 1 2 3 4 5 6 to -3 -2 -1 1 2 3
            n[n>=0]=n[n>=0]+1
        else:
            n=n-num.ceil((len(a)-8)/2)
            
            
        fitFx=lambda x,a:a*x
        paramCharge=curve_fit(fitFx,n,averageChargeLayers[4:-4])
        paramZPos=curve_fit(fitFx,n,aveZpos[4:-4])
        
        averageChargeFitted=fitFx(n,paramCharge[0])
        averageZPosFitted=fitFx(n,paramZPos[0])
        
        lab.plot(averageChargeFitted,averageChargeLayers[4:-4],"or")
        lab.plot(averageChargeFitted,averageChargeFitted,"--")
        #lab.plot(averageZPosFitted,averageChargeFitted,"og")
        
        
        dipolefitted=num.sum(averageChargeFitted*averageZPosFitted)
        diff=[]
        precharge=0
        prez=0
        for counter in range(len(a)-8):
            diff.append((averageChargeFitted[counter]-precharge)*(averageZPosFitted[counter]-prez))
            precharge=averageChargeFitted[counter]
            prez=averageZPosFitted[counter]
            
    
         
        dipole=num.sum(diff[1:])/float(len(diff[1:]))
        print(dipole,dipolefitted)
        
        
        return dipole,dipolefitted
    
        
    except:
        print("Corrupt File")
        
        
# values of k
k=[200,250,300,350,400,450,500,550]
E=[0.5,1,1.5,2,2.5,3]
frameUsed=50
atomNumber=1080 #864
atomPlate=0
frames=100
pathFolder="../Susceptiblity_K_determine/15Layers111Zhou2004/"
plot=False

ke=[[kfile,Efile] for kfile in k for Efile in E]
for fileTag in ke:
    
    filename=("24LayersK_"+str(fileTag[0])+"E_"+str(fileTag[1])+".dump")
    LayerFitDipoleTest(pathFolder+filename,frames,atomNumber,atomPlate,frameUsed)
    

Corrupt File
Corrupt File
Corrupt File
Corrupt File
Corrupt File
Corrupt File
Corrupt File
Corrupt File
Corrupt File
Corrupt File
Corrupt File
Corrupt File
Corrupt File
Corrupt File
Corrupt File
Corrupt File
Corrupt File
Corrupt File
Corrupt File
Corrupt File
Corrupt File
Corrupt File
Corrupt File
Corrupt File
Corrupt File
Corrupt File
Corrupt File
Corrupt File
Corrupt File
Corrupt File
Corrupt File
Corrupt File
Corrupt File
Corrupt File
Corrupt File
Corrupt File
Corrupt File
Corrupt File
Corrupt File
Corrupt File
Corrupt File
Corrupt File
Corrupt File
Corrupt File
Corrupt File
Corrupt File
Corrupt File
Corrupt File
